# MSQ: Necessary Structures

This notebook outlines and creates the necessary code structures to implement the MSQ.

NOTE: this code is written in Python 2.7.x, but all attempts are made to use Python3-compliant syntax. 

In [1]:
# Import relevant libraries
from __future__ import division, print_function
import numpy as np

The following is a very compact description of the method. See the authors' paper for a much more in-depth discussion. 

The key idea is that we define two objects: 

- a set of *quantiles* of some distribution, denote this $\hat{q}$;
- a set of *functions of quantiles*, which map a set of quantiles to a vector of reals. denote this $\hat{\Phi}$. 


